In [162]:
import pandas as pd
from datetime import date, datetime, timedelta
from datetime import date, timedelta


start_date = date(2021, 1, 16)
end_date = date(2021, 11, 18)
delta = end_date - start_date
url = "http://www.nepalstock.com/company/transactions/516/1/?startDate=start_date&endDate=end_date&_limit=10000"

daily_data =  {}

for date in range(delta.days + 1):
    day = start_date + timedelta(days = date)
    if day.weekday() != 5 and day.weekday() != 6:    
        url_day = url.replace("start_date", str(day))
        url_day = url_day.replace("end_date", str(day))
        data = pd.read_html(url_day)[0]
        data.columns = data.iloc[1,:]
        data = data.iloc[2:-1,:]
        if data.size > 0:
            data = data[::-1]
            daily_data[day] = list(map(float, data["Rate"]))

            
print("Data Download Complete")

Data Download Complete


In [167]:
transaction_record = pd.DataFrame(columns = ["Date", "Buy/Sell", "Rate", "Stock", "Cash"])

transact_units = 1000

LTP = 390
TP = 390
value = -TP*transact_units *1.00415-25
position = 1

enter = 0.06
low_buy = 0.033
high_sell = 0.033

def transact(daily_data, transact_date):
    global TP, position, LTP, value, transaction_record     
    for i in daily_data:
        if position == 0 and ((LTP - i)/LTP > enter or (TP - i)/TP > enter):
            position += 1
            value -= transact_units*i*1.00415 - 25
            TP = i
            transaction_record = transaction_record.append({"Date": transact_date, "Buy/Sell": "B", "Rate": i, "Stock" : position*transact_units, "Cash": value }, ignore_index = True)            
        if (TP - i)/TP > low_buy:
            position += 1
            value -= transact_units*i*1.00415 - 25
            TP = i
            transaction_record = transaction_record.append({"Date": transact_date, "Buy/Sell": "B", "Rate": i, "Stock" : position*transact_units, "Cash": value }, ignore_index = True)
        if (i - TP)/TP > high_sell and position > 0:
            value += transact_units*i*0.99585 + 25
            position -= 1
            TP = i
            transaction_record = transaction_record.append({"Date": transact_date, "Buy/Sell": "S", "Rate": i, "Stock" : position*transact_units, "Cash": value }, ignore_index = True)
    LTP = daily_data[-1]

for k,v in daily_data.items():
    transact(v, k)
transaction_record

,Date,Buy/Sell,Rate,Stock,Cash
0,2021-01-18,S,405.0,0,11700.750
1,2021-01-18,B,390.0,1000,-379892.750
2,2021-01-18,S,403.0,0,21459.800
3,2021-01-19,B,389.0,1000,-369129.550
4,2021-01-19,S,409.0,0,38198.100
...,...,...,...,...,...
89,2021-11-02,B,1084.0,1000,-101530.815
90,2021-11-02,S,1120.0,0,1013846.185
91,2021-11-09,B,1083.0,1000,-73623.265
92,2021-11-11,B,1045.0,2000,-1122935.015


In [168]:
profit = transaction_record["Rate"].iloc[-1] * transaction_record["Stock"].iloc[-1] + transaction_record["Cash"].iloc[-1]
print("Total Profit", round(profit))
print("Max Possible Profit", round(transaction_record["Stock"].max()*(transaction_record["Rate"].iloc[-1]-transaction_record["Rate"].iloc[0])))
print("Total Required Cash", round(transaction_record["Rate"].iloc[-1]*transaction_record["Stock"].max()))

Total Profit 1032408
Max Possible Profit 3374500
Total Required Cash 5399500
